In [ ]:
from pathlib import Path
import csv
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import soundfile as sf

In [ ]:
fmc = Path("./FakeMusicCaps/FakeMusicCaps/")

In [ ]:
data_sets = [
    "MusicCaps",
    "MusicGen_medium",
    "audioldm2",
    "musicldm",
    "mustango",
    "stable_audio_open",
]

In [ ]:
import os

for ds in data_sets:
    print(ds)
    # print(len(os.listdir(f'./FakeMusicCaps/{ds}')))
    print(len(os.listdir(fmc / ds)))
    print()

In [ ]:
tempo_shifted_music_caps_folder = Path('./TempoShiftedFakeMusicCaps')
tempo_shifted_music_caps_folder.mkdir(exist_ok=True)

for folder in data_sets:
    folder_path = fmc / folder
    print(f"Writing to {tempo_shifted_music_caps_folder / folder}")
    destination_folder = Path(tempo_shifted_music_caps_folder / folder)
    destination_folder.mkdir(exist_ok=True)
    for song in folder_path.glob('*.wav'):
        y, sr = librosa.load(str(song))
        low, high = 0.8, 1.2
        rate = float((high - low) * np.random.rand() + low)
        new_y = librosa.effects.time_stretch(y=y, rate=rate)
        print(f"\tnew song{song.stem}.wav")
        sf.write(str(destination_folder / f"{song.stem}.wav"), new_y, sr, )
        

In [ ]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib
matplotlib.use('agg')

spectrogram_folder = Path('./TempoShiftedSpectrograms/')
spectrogram_folder.mkdir(exist_ok=True)

for folder in data_sets:
    folder_path = tempo_shifted_music_caps_folder / folder
    destination_folder = Path(spectrogram_folder / folder)
    destination_folder.mkdir(exist_ok=True)
    for music in folder_path.glob('*.wav'):
        y, sr = librosa.load(str(music))
        mel_signal = librosa.feature.melspectrogram(y=y, sr=sr)
        spectrogram = np.abs(mel_signal)
        power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
        fig = plt.figure(figsize=(8, 7))
        canvas = FigureCanvas(fig)
        p = librosa.display.specshow(power_to_db, sr=sr, x_axis='time', y_axis='mel', cmap='magma')
        fig.savefig(destination_folder / f"{music.stem}.png")
        plt.close()

In [ ]:
import gc
gc.collect()

In [ ]:
import os
import shutil

spectrogram_folder = Path('./TempoShiftedSpectrograms/')

binary_spectrogram_folder = Path('./TempoShiftedBinaryClassifiedSpectrograms/')
binary_spectrogram_folder.mkdir(exist_ok=True)

destination_folder = Path(binary_spectrogram_folder / "deepfake")
destination_folder.mkdir(exist_ok=True)

for folder in data_sets:
    if folder != 'SunoCaps' and folder != 'MusicCaps':
        folder_path = spectrogram_folder / folder
        max_samples = len(os.listdir(spectrogram_folder / "MusicCaps")) * 0.2
        count = 0
        for spectrogram in folder_path.glob('*.png'):
            if count > max_samples:
                break
            count += 1
            shutil.copyfile(spectrogram, destination_folder / f"{spectrogram.stem}_{folder}.png")

human_destination = Path(binary_spectrogram_folder / "human")
human_destination.mkdir(exist_ok=True)

folder_path = spectrogram_folder / "MusicCaps"
for spectrogram in folder_path.glob('*.png'):
    shutil.copyfile(spectrogram, human_destination / spectrogram.name)

In [ ]:
for folder in ["human", "deepfake"]:
    print(len(os.listdir(f'./BinaryClassifiedSpectrograms/{folder}')))